# Import

In [1]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv
from ema_workbench.em_framework.evaluators import BaseEvaluator
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import prim
from ema_workbench import Samplers

Then, the model is loaded, and logging is set.

In [2]:
dike_model, planning_steps = get_model_for_problem_formulation(5)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

First, the results of the policy discovery is loaded.

In [3]:
pareto_policies = pd.read_pickle(r'../generated_datasets/initial_Pareto_policies.pkl')

Then, the policies from the analysis are loaded into the model.

In [4]:
policies = []
for row_number in range(pareto_policies.shape[0]):
    policies.append(
        Policy(name=row_number, **pareto_policies.iloc[row_number, :-5].to_dict())
    )

General settings of the model are set.

In [5]:
n_scenarios = int(10_000)

Then, run the model with the robust policies as input, and generate results over a lot of scenarios

In [ ]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios,
                                            policies=policies,
                                            uncertainty_sampling=Samplers.LHS
                                            )

with open(r'../generated_datasets/policy_uncertainty_test.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 10000 scenarios * 45 policies * 1 model(s) = 450000 experiments
  0%|                                  | 163/450000 [00:16<10:34:08, 11.82it/s]

The results are then loaded in for analysis

In [3]:
scenario_results = pd.read_pickle(r'../generated_datasets/policy_uncertainty_test.pkl')
experiments, outcomes = scenario_results
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,48,220.533199,1.0,0.247230,153.375851,10.0,0.997775,178.309323,1.5,0.251558,...,1,5,0,0,5,3,0,0,0,dikesnet
1,21,67.149570,10.0,0.420960,226.487687,1.0,0.209996,202.824278,1.5,0.786272,...,1,5,0,0,5,3,0,1,0,dikesnet
2,31,76.451266,1.0,0.327000,67.115388,1.0,0.876109,182.551213,10.0,0.518388,...,1,5,0,0,5,3,0,2,0,dikesnet
3,61,127.952044,1.0,0.766719,291.080472,1.5,0.371684,107.187029,1.0,0.352222,...,1,5,0,0,5,3,0,3,0,dikesnet
4,121,117.912505,1.0,0.638428,290.598552,1.5,0.397689,159.114245,10.0,0.211288,...,1,5,0,0,5,3,0,4,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66995,4,177.229138,1.5,0.984079,271.295149,1.0,0.478427,195.491156,1.5,0.186950,...,0,2,0,0,7,2,0,995,66,dikesnet
66996,20,302.894975,1.5,0.318097,64.593990,10.0,0.788062,128.320223,1.0,0.207763,...,0,2,0,0,7,2,0,996,66,dikesnet
66997,84,157.204078,1.0,0.083453,253.820279,1.0,0.564167,343.477028,1.5,0.806896,...,0,2,0,0,7,2,0,997,66,dikesnet
66998,71,340.448536,1.0,0.792912,99.445132,10.0,0.602468,97.887405,1.5,0.549740,...,0,2,0,0,7,2,0,998,66,dikesnet


Make a dataframe column for every timestep of the results.

In [ ]:
policies_all_scenarios = pd.DataFrame()
for key, value in outcomes.items():
    temp_df = pd.DataFrame(value, columns=[key + ' '+ str(x) for x in range(len(value[0]))])
    policies_all_scenarios = pd.concat([policies_all_scenarios, temp_df], axis=1)

Add experiments input to the outputs for analysis

In [ ]:
experiments_and_results = pd.concat([experiments,policies_all_scenarios], axis=1)

with open(r'../generated_datasets/policy_uncertainty_experiments_results.pkl', 'wb') as file_pi:
    pickle.dump(experiments_and_results, file_pi)

Combine the variables of different dikes and times

In [ ]:
def combine_columns(dataframe, name):
    combined_columns = []
    for x in dataframe.columns:
        if name in x:
            combined_columns.append(x)

    dataframe['Total ' + str(name)] = dataframe[combined_columns].sum(axis=1)

In [ ]:
combine_columns(policies_all_scenarios, 'Expected Annual Damage')
combine_columns(policies_all_scenarios, 'Dike Investment Costs')
combine_columns(policies_all_scenarios, 'Expected Number of Deaths')
combine_columns(policies_all_scenarios, 'RfR Total Costs')
combine_columns(policies_all_scenarios, 'Expected Evacuation Costs')

Generate statistics

In [ ]:
aggregated_outcomes = pd.concat([experiments[['scenario','policy']],policies_all_scenarios.iloc[:,-5:]], axis=1)

stats_df = pd.DataFrame()
for column in aggregated_outcomes.iloc[:,2:].columns:
    stats_df[column + ' mean'] = aggregated_outcomes.groupby('policy').agg({column:['mean']})
    stats_df[column + ' std'] = aggregated_outcomes.groupby('policy').agg({column:['std']})

with open(r'../generated_datasets/policy_uncertainty_aggregated.pkl', 'wb') as file_pi:
    pickle.dump(aggregated_outcomes, file_pi)